In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)


In [2]:
file = pd.read_csv('sample_text.csv')
file

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [3]:
from sentence_transformers import SentenceTransformer 

In [4]:
transformer = SentenceTransformer('all-mpnet-base-v2')
vectors = transformer.encode(file.text)

In [5]:
vectors.shape

(8, 768)

In [6]:
dir(vectors)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__o

In [7]:
vectors[0]

array([-2.47384887e-03,  3.62672918e-02, -5.29046059e-02, -2.17348486e-02,
        1.41724050e-02, -1.01596001e-03,  2.23636664e-02,  3.65196206e-02,
        3.80901322e-02,  5.22012683e-03, -3.00866067e-02,  2.05449807e-03,
        1.08502693e-02, -8.49087685e-02, -5.57266036e-03, -6.06597960e-02,
        2.32012360e-03, -1.51408892e-02,  1.20671391e-02,  1.98904648e-02,
       -1.27936136e-02, -2.72812098e-02, -3.31878923e-02,  3.40645527e-03,
        8.41170177e-03, -1.94853600e-02,  4.88132425e-02,  2.24119294e-02,
        4.53466959e-02, -5.90892434e-02, -9.13183168e-02, -1.22324598e-03,
       -4.05046009e-02,  8.67169723e-03,  1.26565294e-06, -1.92451756e-02,
       -3.20658535e-02, -4.08172570e-02,  8.79177079e-02, -1.52631355e-02,
        1.09909466e-02, -4.47977819e-02, -4.74821683e-03,  3.52632515e-02,
        1.31071033e-02,  1.09944157e-02, -3.45763774e-03,  9.12350044e-02,
        5.68809435e-02,  4.82893065e-02, -4.51437337e-03,  3.58451493e-02,
       -2.44209338e-02, -

In [8]:
#now we have converted the text to vector we need to attach the faiss index to these vectors the indexs are needed to perform similiarity search
d = vectors.shape[1]
d#d is the dimension which means no of features

768

In [9]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(vectors)                  # add vectors to the index
print(index.ntotal)

True
8


In [22]:
query = 'looking for a place to visit holidays'

In [23]:
xq = transformer.encode(query)
xq.shape

(768,)

In [24]:
import numpy as np
svec = np.array(xq).reshape(1,-1)
svec.shape

(1, 768)

In [ ]:
#with in 8 smples and you want to find the 1 neest neighbors for the first 5 samples
k = 1                          # we want to see 1 earest neighbors
D, I = index.search(vectors[:5], k) # sanity check
print(I)
print(D)                  # neighbors of the 5 last queries

In [25]:
k = 2
D, I = index.search(svec, k)     # actual search

In [26]:
I

array([[6, 7]])

In [27]:
file.loc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
